In [1]:
!pip install selenium

In [2]:
!pip3 install webdriver_manager

In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
import time
import csv

# Setup the WebDriver (using Firefox in this example)
firefox_options = Options()
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=firefox_options)

# Function to extract titles, authors, PMIDs, and citation info for a given page
def get_articles_info_from_page(url):
    articles_info = []
    
    driver.get(url)
    
    # Wait for the page to load
    time.sleep(5)
    
    # Find all the result containers on the current page
    articles = driver.find_elements(By.CSS_SELECTOR, "article.full-docsum")
    
    # Extract the required information from each article
    for article in articles:
        # Extract the title
        title_element = article.find_element(By.CSS_SELECTOR, "a.docsum-title")
        title = title_element.text.strip()
        
        # Extract the authors
        authors_element = article.find_element(By.CSS_SELECTOR, "span.docsum-authors.full-authors")
        authors = authors_element.text.strip()

        # Extract the PMID
        pmid_element = article.find_element(By.CSS_SELECTOR, "span.docsum-pmid")
        pmid = pmid_element.text.strip()

        # Extract the citation
        citation_element = article.find_element(By.CSS_SELECTOR, "span.docsum-journal-citation.full-journal-citation")
        citation = citation_element.text.strip()
        
        # Append the article info
        articles_info.append((title, authors, pmid, citation))
    
    return articles_info

# Loop through the first 100 pages and extract the information
all_articles_info = []
for i in range(1, 51):  # 100 pages
    print(f"Scraping page {i}")
    page_url = f'https://pubmed.ncbi.nlm.nih.gov/?term=neuro&size=200&page={i}'    
    articles_info = get_articles_info_from_page(page_url)
    all_articles_info.extend(articles_info)
    time.sleep(5)  # Sleep between requests to avoid being rate-limited

# Optional: Write the information to a CSV file
with open('pubmed_articles_info_pages_1_to_51.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Authors', 'PMID', 'Citation'])  # Write header
    for info in all_articles_info:
        writer.writerow(info)

# Close the browser
driver.quit()


C:\Users\PC\AppData\Local\Temp\ipykernel_10896\3935413578.py:11: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=firefox_options)


Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Scraping page 11
Scraping page 12
Scraping page 13
Scraping page 14
Scraping page 15
Scraping page 16
Scraping page 17
Scraping page 18
Scraping page 19
Scraping page 20
Scraping page 21
Scraping page 22
Scraping page 23
Scraping page 24
Scraping page 25
Scraping page 26
Scraping page 27
Scraping page 28
Scraping page 29
Scraping page 30
Scraping page 31
Scraping page 32
Scraping page 33
Scraping page 34
Scraping page 35
Scraping page 36
Scraping page 37
Scraping page 38
Scraping page 39
Scraping page 40
Scraping page 41
Scraping page 42
Scraping page 43
Scraping page 44
Scraping page 45
Scraping page 46
Scraping page 47
Scraping page 48
Scraping page 49
Scraping page 50


In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
import time
import csv

# Function to extract provider names, types, specialties, and primary addresses from the given URL
def get_providers_info(driver, url, end_page):
    providers_info = []
    current_page = 1

    driver.get(url)
    time.sleep(5)  # Wait for the initial page to load

    while current_page <= end_page:
        # Get all rows in the table body
        rows = driver.find_elements(By.CSS_SELECTOR, "tbody[role='rowgroup'] > tr[role='row']")
        for row in rows:
            # Get the provider name, type, specialty, and address from each row
            data_elements = row.find_elements(By.CSS_SELECTOR, "td.ds-u-text-align--left")
            name = data_elements[0].text.strip()
            provider_type = data_elements[1].text.strip()
            specialty = data_elements[2].text.strip()
            address = ' '.join(data_elements[3].text.strip().split())  # Format address

            providers_info.append((name, provider_type, specialty, address))

        # Logic to click on the next page button
        try:
            # Use JavaScript to click the next page button
            next_page_button = driver.find_element(By.XPATH, "//button[contains(@aria-label, 'Next Page') and not(@disabled)]")
            driver.execute_script("arguments[0].click();", next_page_button)
            current_page += 1
            time.sleep(5)  # Wait for the next page to load
        except NoSuchElementException:
            print("No more pages to scrape.")
            break
        except Exception as e:
            print(f"An error occurred: {e}")
            break

        print(f"Scraped page {current_page}")

    return providers_info

# Setup the WebDriver (using Firefox in this example)
firefox_options = Options()
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=firefox_options)

# Define the URL to scrape
base_url = 'https://openpaymentsdata.cms.gov/search?searchType=Provider&FirstName=&LastName=&Country=UNITED+STATES&State=IN&City=&Zip=&Specialty=&Type=Covered+Recipient+Physician&NPI='

# Define the last page number to scrape
end_page = 2070  # Adjust this to the actual number of pages you wish to scrape

# Get provider information from the pages
providers_list = get_providers_info(driver, base_url, end_page)

# Optional: Write the provider information to a CSV file
with open('providers_info.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Provider Name', 'Provider Type', 'Specialty', 'Primary Address'])  # Write header
    for info in providers_list:
        writer.writerow(info)

# Close the WebDriver
driver.quit()

# Print a message when the script is finished
print("Scraping complete, data written to providers_info.csv")


C:\Users\PC\AppData\Local\Temp\ipykernel_17000\4096725280.py:50: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=firefox_options)


Scraped page 2
Scraped page 3
Scraped page 4
Scraped page 5
Scraped page 6
Scraped page 7
Scraped page 8
Scraped page 9
Scraped page 10
Scraped page 11
Scraped page 12
Scraped page 13
Scraped page 14
Scraped page 15
Scraped page 16
Scraped page 17
Scraped page 18
Scraped page 19
Scraped page 20
Scraped page 21
Scraped page 22
Scraped page 23
Scraped page 24
Scraped page 25
Scraped page 26
Scraped page 27
Scraped page 28
Scraped page 29
Scraped page 30
Scraped page 31
Scraped page 32
Scraped page 33
Scraped page 34
Scraped page 35
Scraped page 36
Scraped page 37
Scraped page 38
Scraped page 39
Scraped page 40
Scraped page 41
Scraped page 42
Scraped page 43
Scraped page 44
Scraped page 45
Scraped page 46
Scraped page 47
Scraped page 48
Scraped page 49
Scraped page 50
Scraped page 51
Scraped page 52
Scraped page 53
Scraped page 54
Scraped page 55
Scraped page 56
Scraped page 57
Scraped page 58
Scraped page 59
Scraped page 60
Scraped page 61
Scraped page 62
Scraped page 63
Scraped page 64
